In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tslearn.metrics import dtw

In [3]:
data_small = pd.read_csv('3months_under10_000.csv',index_col = 'day', parse_dates = True)

C:\Users\nakae\AppData\Local\Temp\ipykernel_24700\4100203135.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_small = pd.read_csv('3months_under10_000.csv',index_col = 'day', parse_dates = True)


In [241]:
data_small.head()

,day,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,...,day_firstseen,hour_firstseen,month_firstseen,month_lastseen,day_lastseen,hour_lastseen,day_created,month_created,day_expires,month_expires
0,2023-01-12,popebogvo,event.api.drift.com,drift.com,1,4,2023-01-12 15:38:33.427,2023-01-12 15:38:33.427,0,17,...,3,15,1,1,3,15,5.0,4.0,1.0,4.0
1,2022-12-20,prurkofbu,bootstrap.api.drift.com,drift.com,1,4,2022-12-20 20:17:38.839,2022-12-20 20:17:38.839,0,9,...,1,20,12,12,1,20,5.0,4.0,1.0,4.0
2,2022-12-30,shwedvervi,presence.api.drift.com,drift.com,1,4,2022-12-30 15:50:29.299,2022-12-30 15:50:29.299,0,6,...,4,15,12,12,4,15,5.0,4.0,1.0,4.0
3,2023-01-12,volcagbuca,presence.api.drift.com,drift.com,3,80,2023-01-12 00:09:47.035,2023-01-12 21:59:53.949,0,17,...,3,0,1,1,3,21,5.0,4.0,1.0,4.0
4,2022-12-20,shwedvervi,targeting.api.drift.com,drift.com,1,12,2022-12-20 12:17:32.930,2022-12-20 21:45:38.544,0,9,...,1,12,12,12,1,21,5.0,4.0,1.0,4.0


In [232]:
data_small.whois.fillna('unknown',inplace=True)

In [233]:
# create a timeseen column
data_small['timelastseen'] = pd.to_datetime(data_small['timelastseen'])

In [234]:
data_small['timefirstseen'] = pd.to_datetime(data_small['timefirstseen'])

In [236]:
data_small['timeseen'] = (data_small['timelastseen']-data_small['timefirstseen'])

In [240]:
data_small['timeseen'] = data_small['timeseen'].dt.total_seconds()

AttributeError: Can only use .dt accessor with datetimelike values

### Find and create an anomaly column with Finns method

In [9]:
#organize data like in linear regression example
counts = data_small.pivot_table(values = 'num_clients',index = data_small.index.date,
                         columns = 'dns_qdomainname',
                         aggfunc=sum,
                         fill_value = 0)

In [10]:
#normalize data
counts.max().sort_values(ascending=False).to_csv('counts.csv')
counts = counts/counts.max()

In [11]:
#create domians by checking similairty to 2 anomalies
domains = []
dns = data_small.dns_qdomainname.unique()
similarity_slider = .6
#numba this
for i in range (len(dns)-1):
    val = dtw(counts['beside.media'],counts[dns[i]])
    if (val <= similarity_slider) & (dtw(counts['123w0w.com'],counts[dns[i]]) <= similarity_slider):
        domains.append(dns[i])

In [64]:
domains

['servogram.io',
 'band.us',
 'calia.com',
 'wcoanimedub.tv',
 'armakeup.com',
 'safelifedefense.com',
 'boardgamegeek.com',
 'avail.co',
 'sbrchamber.com',
 'nkpmedical.com',
 'babbel.com',
 'needtoknowfacts.com',
 'forfamilymall.com',
 'andimejia.com',
 'magnus.ms',
 'ttlstatic.com',
 'lh.social',
 'roundme.com',
 'history.com',
 'as.me',
 'bizorgraphics.com',
 'bbcselect.com',
 'beautiful.ai',
 'danmarketplace.com',
 'amcplus.com',
 'smallseotools.com',
 'tango.us',
 'cole.ca',
 'pngjoy.com',
 'encentivizer.com',
 'unnumelom.com',
 'iott.cc',
 'aglowlifestyle.com',
 'undark.org',
 'jar-download.com',
 'donjohnston.net',
 'mmsend50.com',
 'proquest.com',
 'segreencolumn.com',
 'open.ai',
 'wsktbf.net',
 'manilarepublic.com',
 'ticketschool.com',
 'morriscenter.org',
 'lbcgroup.tv',
 'itworkscdn.net',
 'adoptopenjdk.net',
 'loadcomplete.com',
 'them.us',
 'updpopcorntime.xyz',
 'raab3frog.com',
 'yale.edu',
 'static-carhp.com',
 'dtra.mil',
 'dakasapps.com',
 'kidsempoweredonthemove.o

In [21]:
data_small.loc[:,'anomaly_2'] = 0
data_small.loc[data_small.dns_qdomainname.isin(domains),'anomaly_2'] = 1

In [22]:
data_small.head()

,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois,timeseen,anomaly_2,tld,mal_tld
day,,,,,,,,,,,,,,,,,,,
2023-01-04,enilokmacy,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-10,enilokmacy,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-09,fecjavfec,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-20,fecjavfec,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 10:46:16.076000,0,com,0
2023-01-20,oreethecaw,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 11:34:16.222000,0,com,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-21,thwijijod,cdn-prod-sg.yepgarage.info,yepgarage.info,1,20,2022-12-21 05:28:15.901,2022-12-21 05:28:15.901,0,1,0.0,yepgarage.info,2020-11-19,2020-11-19,2021-11-19,whois.gandi.net,0 days 00:00:00,0,info,0
2023-01-12,thwijijod,www.omaemirates.com,omaemirates.com,1,1,2023-01-12 06:01:10.576,2023-01-12 06:01:10.576,0,1,0.0,omaemirates.com,1998-10-28,1998-10-28,2020-10-27,whois.networksolutions.com,0 days 00:00:00,0,com,0
2023-01-03,volcagbuca,www.copenhagen.com,copenhagen.com,2,2,2023-01-03 01:18:13.956,2023-01-03 01:18:13.983,0,1,0.0,copenhagen.com,1996-04-02,1996-04-02,2026-04-02,whois.godaddy.com,0 days 00:00:00.027000,0,com,0


### Create a column of tld's and whether or not it is known to be malicious

In [23]:
# tld column
data_small['tld'] = data_small.dns_qdomainname.str.split('.').str[1]

In [24]:
# column of tlds cross referenced with a list of the top tlds used in malicious activity
mal_tlds = ['live','monster','okinawa','top','boutique','beauty','cn','zone','autos','zw','bd','ke','am','sbs','date','pw','quest','cd','bid','cyou','ml','ws','icu','stream','email','uno','cam','xyz','tokyo','help','casa','rest','win']
data_small.loc[:,'mal_tld']=0
data_small.loc[data_small.tld.isin(mal_tlds),'mal_tld']=1
data_small.mal_tld.sum()

36736

In [27]:
# of the potentialy anomalys using anomaly finder 2, how many fall into the list of malicious tld
domains
domains_tld=[i.split('.')[1] for i in domains]
domains_mal = set(domains_tld) & set(mal_tlds)
domains_mal

{'cn', 'email', 'live', 'top', 'xyz'}

In [28]:
# anomaly finder 1?
anomaly1 = data_small.loc[data_small.anomaly==1].dns_qdomainname.unique().tolist()
anomaly1_tld = [i.split('.')[1] for i in domains]
domains1_mal = set(anomaly1_tld) & set(mal_tlds)
domains1_mal

{'cn', 'email', 'live', 'top', 'xyz'}

### Fixing the bad data in the created, updated, expired columns

In [76]:
# get the indices of all bad times
for i in range(len(data_small)):
    try:
        pd.to_datetime(data_small.expires[i])
    except:
        print(i)


7017679
7017680
7017681
7017682
7017683
7017684
7017685
7017686
7017687
7017688
7017689
7017690
7017691
7017692
7017693
7017694
7017695
7017696
7017697
7017698
7017699
7017700
7017701
7017702
7017703
7017704
7017705
7017706
7017707
7017708
7336087
7336088
7336089
7336090
7336091
7336092
7336093
7336094
7336095
7336096
7336097
7336098
7336099
7336100
7336101
7336102
7336103
7336104
7336105
7336106
7336107
7336108
7336109
7336110
7336111
7336112
7336113
7336114
7336115
7336116
7336117
7336118
7336119
7336120
7336121
7336122
7336123
7336124
7336125
7336126
7336127
7336128
7336129
7336130
7336131
7336132
7336133
7336134
7336135
7336136
7336137
7336138
7336139
7336140
7336141
7336142
7336143
7336144
7336145
7336146
7336147
7336148
7336149
7336150
7336151
7336152
7336153
7336154
7336155
7628174
7628175
7628176
7628177
7628178
7628179
7628180
7628181
7628182
7628183
7628184
7628185
7628186
7628187
7647656
7647657
7647658
7647659
7647660
7647661
7647662
7647663
7647664
7647665
7647666
7647667


In [77]:
badtime_e = [7017679,
7017680,
7017681,
7017682,
7017683,
7017684,
7017685,
7017686,
7017687,
7017688,
7017689,
7017690,
7017691,
7017692,
7017693,
7017694,
7017695,
7017696,
7017697,
7017698,
7017699,
7017700,
7017701,
7017702,
7017703,
7017704,
7017705,
7017706,
7017707,
7017708,
7336087,
7336088,
7336089,
7336090,
7336091,
7336092,
7336093,
7336094,
7336095,
7336096,
7336097,
7336098,
7336099,
7336100,
7336101,
7336102,
7336103,
7336104,
7336105,
7336106,
7336107,
7336108,
7336109,
7336110,
7336111,
7336112,
7336113,
7336114,
7336115,
7336116,
7336117,
7336118,
7336119,
7336120,
7336121,
7336122,
7336123,
7336124,
7336125,
7336126,
7336127,
7336128,
7336129,
7336130,
7336131,
7336132,
7336133,
7336134,
7336135,
7336136,
7336137,
7336138,
7336139,
7336140,
7336141,
7336142,
7336143,
7336144,
7336145,
7336146,
7336147,
7336148,
7336149,
7336150,
7336151,
7336152,
7336153,
7336154,
7336155,
7628174,
7628175,
7628176,
7628177,
7628178,
7628179,
7628180,
7628181,
7628182,
7628183,
7628184,
7628185,
7628186,
7628187,
7647656,
7647657,
7647658,
7647659,
7647660,
7647661,
7647662,
7647663,
7647664,
7647665,
7647666,
7647667,
7647668,
7647669,
7647670,
7647671,
7647672,
7647673,
7647674,
7647675,
7647676,
7647677,
7647678,
7647679,
7647680,
7647681,
7647682,
7647683,
7647684,
7647685,
7647686,
7647687,
7647688,
7647689,
7647690,
7647691,
7647692,
7647693,
7647694,
7647695,
7647696,
7647697,
7647698,
7647699,
7647700,
7647701,
7647702,
7647703,
7647704,
7647705,
7647706,
7647707,
7647708,
7794828,
7794829,
7794830,
7794831,
7794832,
7794833,
7794834,
7794835,
7794836,
7794837,
7846575,
7846576,
7846577,
7846578,
7846579,
7846580,
7846581,
7846582,
7846583,
7846584,
7853177,
7903029,
7938764,
7938765,
7938766,
7974129,
7974130,
7974131,
7974132,
7974133,
7974134,
7991433,
7991434,
8021368,
8021369,
8021370,
8021371,
8021372,
8021373,
8021374,
8021375,
8021376,
8021377,
8021378,
8021379,
8021380,
8048752,
8048753,
8048754,
8048755,
8048756,
8048757,
8057153,
8069148,
8078112,
8116554,
8116555,
8135395,
8138031,
8141359,
8160724,
8160725,
8179615,
8189456,
8193134,
8203184,
8203185,
8205323,
8205449,
8205450,
8208303,
8222717,
8222718,
8224212,
8227575,
8229089,
8257853,
8264014]

In [72]:
badtime=[7017679,
7017680,
7017681,
7017682,
7017683,
7017684,
7017685,
7017686,
7017687,
7017688,
7017689,
7017690,
7017691,
7017692,
7017693,
7017694,
7017695,
7017696,
7017697,
7017698,
7017699,
7017700,
7017701,
7017702,
7017703,
7017704,
7017705,
7017706,
7017707,
7017708,
7336087,
7336088,
7336089,
7336090,
7336091,
7336092,
7336093,
7336094,
7336095,
7336096,
7336097,
7336098,
7336099,
7336100,
7336101,
7336102,
7336103,
7336104,
7336105,
7336106,
7336107,
7336108,
7336109,
7336110,
7336111,
7336112,
7336113,
7336114,
7336115,
7336116,
7336117,
7336118,
7336119,
7336120,
7336121,
7336122,
7336123,
7336124,
7336125,
7336126,
7336127,
7336128,
7336129,
7336130,
7336131,
7336132,
7336133,
7336134,
7336135,
7336136,
7336137,
7336138,
7336139,
7336140,
7336141,
7336142,
7336143,
7336144,
7336145,
7336146,
7336147,
7336148,
7336149,
7336150,
7336151,
7336152,
7336153,
7336154,
7336155,
7628174,
7628175,
7628176,
7628177,
7628178,
7628179,
7628180,
7628181,
7628182,
7628183,
7628184,
7628185,
7628186,
7628187,
7846575,
7846576,
7846577,
7846578,
7846579,
7846580,
7846581,
7846582,
7846583,
7846584,
7853177,
7903029,
7938764,
7938765,
7938766,
7974129,
7974130,
7974131,
7974132,
7974133,
7974134,
7991433,
7991434,
8021368,
8021369,
8021370,
8021371,
8021372,
8021373,
8021374,
8021375,
8021376,
8021377,
8021378,
8021379,
8021380,
8048752,
8048753,
8048754,
8048755,
8048756,
8048757,
8057153,
8069148,
8078112,
8094647,
8116554,
8116555,
8135395,
8138031,
8141359,
8160724,
8160725,
8179615,
8189456,
8193134,
8203184,
8203185,
8205323,
8205449,
8205450,
8208303,
8222717,
8222718,
8224212,
8227575,8229089,8257853,8264014]

In [143]:
all_badtime = list(set(badtime) | set(badtime_e))
all_badtime

[8205323,
 8069148,
 8193134,
 8048752,
 8048753,
 8048754,
 8048755,
 8048756,
 8048757,
 7853177,
 7991433,
 7991434,
 8205449,
 8205450,
 7794828,
 7794829,
 7794830,
 7794831,
 7794832,
 7794833,
 7794834,
 7794835,
 7794836,
 7794837,
 7336087,
 7336088,
 7336089,
 7336090,
 7336091,
 7336092,
 7336093,
 7336094,
 7336095,
 7336096,
 7336097,
 7336098,
 7336099,
 7336100,
 7336101,
 7336102,
 7336103,
 7336104,
 7336105,
 7336106,
 7336107,
 7336108,
 7336109,
 7336110,
 7336111,
 7336112,
 7336113,
 7336114,
 7336115,
 7336116,
 7336117,
 7336118,
 7336119,
 7336120,
 7336121,
 7336122,
 7336123,
 7336124,
 7336125,
 7336126,
 7336127,
 7336128,
 7336129,
 7336130,
 7336131,
 7336132,
 7336133,
 7336134,
 7336135,
 7336136,
 7336137,
 7336138,
 7336139,
 7336140,
 7336141,
 7336142,
 7017679,
 7017680,
 7017681,
 7017682,
 7017683,
 7017684,
 7017685,
 7017686,
 7017687,
 7017688,
 7017689,
 7017690,
 7017691,
 7017692,
 7017693,
 7017694,
 7017695,
 7017696,
 7017697,
 7017698,


In [129]:
badtime_dates = data_small.index[all_badtime]
badtime_cols = data_small.iloc[all_badtime,:]

set()

In [130]:
# are any of the domains with bad times in our list projected anomalies?
badtime_doms = badtime_cols.dns_qdomainname.unique()
set(badtime_doms)&set(domains)

set()

In [156]:
badtime_dates

DatetimeIndex(['2022-12-29', '2023-01-11', '2022-12-12', '2022-12-13',
               '2023-01-04', '2023-01-20', '2023-01-23', '2023-01-18',
               '2022-12-12', '2022-12-18',
               ...
               '2022-12-06', '2023-01-23', '2023-01-03', '2023-01-16',
               '2023-01-09', '2023-01-16', '2023-01-16', '2023-01-23',
               '2022-12-08', '2022-12-08'],
              dtype='datetime64[ns]', name='day', length=245, freq=None)

In [158]:
data_small.drop(badtime_dates,axis=0,inplace=True)

In [159]:
data_small['created'] = pd.to_datetime(data_small['created'])
data_small['updated'] = pd.to_datetime(data_small['updated'])
data_small['expires'] = pd.to_datetime(data_small['expires'])

In [166]:
data_small['time_existed'] = data_small['expires'] - data_small['created']
data_small['time_existed']

day
2023-01-12   10594 days
2022-12-20   10594 days
2022-12-30   10594 days
2023-01-12   10594 days
2022-12-20   10594 days
                ...    
2022-12-11     365 days
2022-12-30    1096 days
2022-12-27    1096 days
2023-01-01    4018 days
2023-01-12    8035 days
Name: time_existed, Length: 849420, dtype: timedelta64[ns]

In [208]:
data_small['time_existed'] = data_small.time_existed.str.split(' ').str[0]
data_small.rename(columns = {'time_existed':'days_existed'}, inplace = True)

In [222]:
data_small['days_existed'] = data_small.days_existed.astype(pd.Int64Dtype())

### convert timefirstseen, timelastseen, created, expires columns to numeric values

In [176]:
data_small['month_firstseen'] = data_small.timefirstseen.dt.month
data_small['day_firstseen'] = data_small.timefirstseen.dt.weekday
data_small['hour_firstseen'] = data_small.timefirstseen.dt.hour
data_small['month_lastseen'] = data_small.timelastseen.dt.month
data_small['day_lastseen'] = data_small.timelastseen.dt.weekday
data_small['hour_lastseen'] = data_small.timelastseen.dt.hour
data_small['day_created'] = data_small.created.dt.weekday
data_small['month_created'] = data_small.created.dt.month
data_small['day_expires'] = data_small.expires.dt.weekday
data_small['month_expires'] = data_small.expires.dt.month

In [248]:
#data_small.to_csv('data_small_update.csv')

## Random Forest Classifier

In [242]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.impute import SimpleImputer

In [243]:
data_small.head()

,day,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,...,day_firstseen,hour_firstseen,month_firstseen,month_lastseen,day_lastseen,hour_lastseen,day_created,month_created,day_expires,month_expires
0,2023-01-12,popebogvo,event.api.drift.com,drift.com,1,4,2023-01-12 15:38:33.427,2023-01-12 15:38:33.427,0,17,...,3,15,1,1,3,15,5.0,4.0,1.0,4.0
1,2022-12-20,prurkofbu,bootstrap.api.drift.com,drift.com,1,4,2022-12-20 20:17:38.839,2022-12-20 20:17:38.839,0,9,...,1,20,12,12,1,20,5.0,4.0,1.0,4.0
2,2022-12-30,shwedvervi,presence.api.drift.com,drift.com,1,4,2022-12-30 15:50:29.299,2022-12-30 15:50:29.299,0,6,...,4,15,12,12,4,15,5.0,4.0,1.0,4.0
3,2023-01-12,volcagbuca,presence.api.drift.com,drift.com,3,80,2023-01-12 00:09:47.035,2023-01-12 21:59:53.949,0,17,...,3,0,1,1,3,21,5.0,4.0,1.0,4.0
4,2022-12-20,shwedvervi,targeting.api.drift.com,drift.com,1,12,2022-12-20 12:17:32.930,2022-12-20 21:45:38.544,0,9,...,1,12,12,12,1,21,5.0,4.0,1.0,4.0


In [244]:
data_small.columns

Index(['day', 'shieldid', 'dns_qname', 'dns_qdomainname', 'num_clients',
       'num_responses', 'timefirstseen', 'timelastseen', 'anomaly',
       'shieldid_count', 'rank', 'domain', 'created', 'updated', 'expires',
       'whois', 'timeseen', 'anomaly_2', 'tld', 'mal_tld', 'days_existed',
       'day_firstseen', 'hour_firstseen', 'month_firstseen', 'month_lastseen',
       'day_lastseen', 'hour_lastseen', 'day_created', 'month_created',
       'day_expires', 'month_expires'],
      dtype='object')

In [245]:
X = data_small.drop(['day','anomaly','anomaly_2','dns_qname','dns_qdomainname','timefirstseen','timelastseen','domain','created','updated','expires'],axis=1)
y = data_small['anomaly_2']

In [255]:
data_small.isna().sum()

day                    0
shieldid               0
dns_qname              0
dns_qdomainname        0
num_clients            0
num_responses          0
timefirstseen          0
timelastseen           0
anomaly                0
shieldid_count         0
rank                   0
domain                 0
created             9499
updated             9499
expires            16026
whois                  0
timeseen               0
anomaly_2              0
tld                    0
mal_tld                0
days_existed       16049
day_firstseen          0
hour_firstseen         0
month_firstseen        0
month_lastseen         0
day_lastseen           0
hour_lastseen          0
day_created         9499
month_created       9499
day_expires        16026
month_expires      16026
dtype: int64

In [271]:
X.columns

Index(['shieldid', 'num_clients', 'num_responses', 'shieldid_count', 'rank',
       'whois', 'timeseen', 'tld', 'mal_tld', 'days_existed', 'day_firstseen',
       'hour_firstseen', 'month_firstseen', 'month_lastseen', 'day_lastseen',
       'hour_lastseen', 'day_created', 'month_created', 'day_expires',
       'month_expires'],
      dtype='object')

In [284]:
numerical_feats = ['num_clients', 'num_responses','shieldid_count','rank','timeseen',
                   'days_existed','day_firstseen','hour_firstseen','month_firstseen', 
                   'month_lastseen', 'day_lastseen','hour_lastseen', 'day_created',
                   'month_created', 'day_expires','month_expires']
categorical_feats = ['shieldid','whois','tld','mal_tld']

In [285]:
num_processor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler', StandardScaler())  
])

cat_processor = Pipeline(steps=[
     ('imputer', SimpleImputer(strategy = 'constant')),
     ('encoder', OneHotEncoder())   
 ])

In [286]:
feat_processor = ColumnTransformer(transformers = [
    ('num_pipe', num_processor,numerical_feats),
    ('cat_pipe', cat_processor, categorical_feats)
],remainder='drop')

In [287]:
RF= RandomForestClassifier()

In [288]:
pipe = Pipeline(steps=[
    ('feature_processor', feat_processor),
    ('clf', RF)
])
pipe

Pipeline(steps=[('feature_processor',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['num_clients',
                                                   'num_responses',
                                                   'shieldid_count', 'rank',
                                                   'timeseen', 'days_existed',
                                                   'day_firstseen',
                                                   'hour_firstseen',
                                                   'month_firstseen',
                                                   'month_lastseen',
                                                   'day_lastseen',
                                                   'hour_lastseen',
                                                   'day_created',
                                                   'month_created',
                                                   'day_expires',
                                                   'month_expires']),
                                                 ('cat_pipe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['shieldid', 'whois', 'tld',
                                                   'mal_tld'])])),
                ('clf', RandomForestClassifier())])

In [289]:
pipe.fit(X,y)

Pipeline(steps=[('feature_processor',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['num_clients',
                                                   'num_responses',
                                                   'shieldid_count', 'rank',
                                                   'timeseen', 'days_existed',
                                                   'day_firstseen',
                                                   'hour_firstseen',
                                                   'month_firstseen',
                                                   'month_lastseen',
                                                   'day_lastseen',
                                                   'hour_lastseen',
                                                   'day_created',
                                                   'month_created',
                                                   'day_expires',
                                                   'month_expires']),
                                                 ('cat_pipe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['shieldid', 'whois', 'tld',
                                                   'mal_tld'])])),
                ('clf', RandomForestClassifier())])